In [2]:
from common import *
%matplotlib inline
canvas(16, 9)

/home/ec2-user/anaconda3/lib/python3.6/site-packages/pandas/core/common.py:701: RuntimeWarning: divide by zero encountered in log
  return func(obj, *args, **kwargs)


In [3]:
o = O()
def_W(o)
def_Y(o, include='scl std')
def_X(o, include='wqtl wnrm')

/home/ec2-user/Notebooks/g-research-ffc2018/helper/weighted.ipynb:11: RuntimeWarning: invalid value encountered in true_divide
  "import notebook_import_hook\n",
/home/ec2-user/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1901: RuntimeWarning: invalid value encountered in greater
  cond1 = (0 < q) & (q < 1)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1901: RuntimeWarning: invalid value encountered in less
  cond1 = (0 < q) & (q < 1)


In [4]:
dd = dh
###

In [5]:
Y = o.Yscl
W = o.Wscl
###
y = Y.makeil_like(dd)
w = W.makeil_like(dd)

In [6]:
Xxsgnintwqtla = pd.read_hdf('/big/data/features/sw32/xx.log.zc(-93).mintco0[4,2].wqtla')
Xxsgnintwnrma = pd.read_hdf('/big/data/features/sw32/xx.log.zc(-93).mintco0[4,2].wnrma')
X3sgnintwqtla = pd.read_hdf('/big/data/features/sw32/x3.log.zc(-93).mintco0[4,2].wqtla')
X3sgnintwnrma = pd.read_hdf('/big/data/features/sw32/x3.log.zc(-93).mintco0[4,2].wnrma')

In [20]:
#X = pd.concat([o.Xwqtla, Xxsgnintwqtla, X3sgnintwqtla], axis=1)
X = pd.concat([o.Xwnrma, Xxsgnintwnrma, X3sgnintwnrma], axis=1)
Xpos = X.clip(lower=0).colname_append('.pos', level=0)
Xneg = X.clip(upper=0).colname_append('.neg', level=0)
X = pd.concat([Xpos, Xneg], axis=1)
#feats = xfeats
###
x = X.makeil_like(dd) #feature_filter(, feats)
y0 = o.Y_wmean.makeil_like(dd).fillna(0)
from sklearn.preprocessing import PolynomialFeatures as PF
#pf = PF(degree=2, include_bias=False)
#xp = pf.fit_transform(x)
#xf = pd.DataFrame(xp, columns=pf.get_feature_names(x.columns), index=x.index, copy=False)
xf = x
xf.loc[:, 'y0'] = y0
assert ~xf.y0.isna().any()

In [10]:
G = pd.read_hdf(top_dir + 'data/analysis/G.m3.hdf5')
group = pd.read_hdf(top_dir + 'data/analysis/group.m3.hdf5')

In [11]:
list(enumerate(G.columns))

[(0, 'm3_BigBrother'),
 (1, 'm3_BoneGrowth'),
 (2, 'm3_Collarbone'),
 (3, 'm3_LargeSiamese'),
 (4, 'm3_LittleBrother'),
 (5, 'm3_MiddleSiamese'),
 (6, 'm3_Sidebar'),
 (7, 'm3_SmallSeparate'),
 (8, 'm3_SmallSiamese')]

In [8]:
with open(top_dir + 'data/chance/days_order.txt', 'r') as file:
    days = pd.Series([int(x) for x in file.read().splitlines()], name='Day')
n_days = len(days)
#days_a, days_b, days_c = days[:n_days // 3], days[n_days // 3:(2 * n_days) // 3], days[(2 * n_days) // 3:]
days_a, days_b = set(days[:n_days // 2]), set(days[n_days // 2:])
#tr, cv, ep = df.Day.isin(days_a), df.Day.isin(days_b), df.Day.isin(days_c)
tr, cv = dh.Day.isin(days_a), dh.Day.isin(days_b)

In [23]:
# wnrm
import gc
gc.collect()
tr, cv = dh.Day.isin(days_a), dh.Day.isin(days_b)
M = 1
GRP = None
tr = tr & (dh.Market == M) #(group == GRP).makeil_like(dh) #
cv = cv & (dh.Market == M) #(group == GRP).makeil_like(dh) #
from sklearn.linear_model import LinearRegression as LR
lr = LR(fit_intercept=True)
lr = lr.fit(xf[tr], y[tr], sample_weight=w[tr])
yf = lr.predict(xf)
print(
    ((yf[tr] - y[tr]) ** 2 * w[tr]).sum() / (y[tr] ** 2 * w[tr]).sum(),
    ((yf[cv] - y[cv]) ** 2 * w[cv]).sum() / (y[cv] ** 2 * w[cv]).sum(),
)

0.376257762882 0.589107411074


In [ ]:
sym 0.421374194893 0.54639807197

In [12]:
# wnrm
for i in range(G.columns.shape[0]):
    tr, cv = dh.Day.isin(days_a), dh.Day.isin(days_b)
    M = 3
    GRP = i
    tr = tr & (group == GRP).makeil_like(dh) #(dh.Market == M)
    cv = cv & (group == GRP).makeil_like(dh) #(dh.Market == M)
    #
    import gc
    gc.collect()
    #
    from sklearn.linear_model import LinearRegression as LR
    lr = LR(fit_intercept=True)
    lr = lr.fit(xf[tr], y[tr], sample_weight=w[tr])
    yf = lr.predict(xf)
    print(i, G.columns[i],
        ((yf[tr] - y[tr]) ** 2 * w[tr]).sum() / (y[tr] ** 2 * w[tr]).sum(),
        ((yf[cv] - y[cv]) ** 2 * w[cv]).sum() / (y[cv] ** 2 * w[cv]).sum(),
    )

0 m3_BigBrother 0.570844225087 0.890452335603
1 m3_BoneGrowth 0.124187988975 4.5853034462
2 m3_Collarbone 0.542329946492 0.773125233913
3 m3_LargeSiamese 0.742537652131 0.916645089696
4 m3_LittleBrother 0.650601728772 1.10815566747
5 m3_MiddleSiamese 0.725172847577 0.884153377544
6 m3_Sidebar 0.632588224462 0.963618763065
7 m3_SmallSeparate 0.25385011465 2.15071819262
8 m3_SmallSiamese 0.659858911959 0.785714061138


In [143]:
combos.mintco0(4, 4, 2)

array([[ 1, -1,  0,  0],
       [ 1,  0, -1,  0],
       [ 1,  0,  0, -1],
       [ 0,  1, -1,  0],
       [ 0,  1,  0, -1],
       [ 0,  0,  1, -1],
       [ 1, -2,  1,  0],
       [ 1,  1, -2,  0],
       [ 2, -1, -1,  0],
       [ 1, -2,  0,  1],
       [ 1,  1,  0, -2],
       [ 2, -1,  0, -1],
       [ 1,  0, -2,  1],
       [ 1,  0,  1, -2],
       [ 2,  0, -1, -1],
       [ 0,  1, -2,  1],
       [ 0,  1,  1, -2],
       [ 0,  2, -1, -1],
       [ 1, -2, -1,  2],
       [ 1, -2,  2, -1],
       [ 1, -1, -2,  2],
       [ 1, -1, -1,  1],
       [ 1, -1,  1, -1],
       [ 1, -1,  2, -2],
       [ 1,  1, -1, -1],
       [ 1,  2, -2, -1],
       [ 1,  2, -1, -2],
       [ 2, -2, -1,  1],
       [ 2, -2,  1, -1],
       [ 2, -1, -2,  1],
       [ 2, -1,  1, -2],
       [ 2,  1, -2, -1],
       [ 2,  1, -1, -2]], dtype=int8)